In [17]:
import scanpy as sc
import pandas as pd
import os

# Load your AnnData object
adata = sc.read_h5ad('/home/glennrdx/Documents/Research_Project/processed_h5ad/crypt_enriched_final.h5ad')

# Define the directory path where the CSVs will be saved
output_dir = "/home/glennrdx/Documents/Research_Project/scRNAseq-MSc-Analysis/upstream_analysis/crypt/differential_expression_scanpy"

# Ensure the directory exists (create it if it doesn't)
os.makedirs(output_dir, exist_ok=True)

# Get the unique cluster identifiers
leiden15 = adata.obs['leiden15'].unique()
results = {}

# Perform differential expression analysis for each cluster
for cluster in leiden15:
    # Subset the data for the cluster
    adata_cluster = adata[adata.obs['leiden15'] == cluster]

    # Perform differential expression between Diet conditions
    sc.tl.rank_genes_groups(adata_cluster, groupby='Diet', method='t-test')  # or 'wilcoxon', 'logreg', etc.

    # Store the results
    results[cluster] = adata_cluster.uns['rank_genes_groups']

    # Unpack the results
    genes = results[cluster]['names']
    pvals = results[cluster]['pvals']
    logfoldchanges = results[cluster]['logfoldchanges']

    # Flatten the tuples and create the DataFrame
    flat_results = []
    for i in range(len(genes)):
        for j in range(len(genes[i])):  # iterate over each item in the tuple
            flat_results.append({
                'gene': genes[i][j],
                'p.value': pvals[i][j],
                'logfc': logfoldchanges[i][j]
            })
    
    # Convert to DataFrame
    df = pd.DataFrame(flat_results)

    # Define the file path for the current cluster
    file_path = os.path.join(output_dir, f'differential_expression_cluster_{cluster}.csv')
    
    # Save the DataFrame to a CSV file with the specified format
    df.to_csv(file_path, index=False)
    
    print(f"Saved results for cluster {cluster} to {file_path}")


/home/glennrdx/virtual_envs/.venv/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:645: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


Saved results for cluster Enterocyte Progenitor to /home/glennrdx/Documents/Research_Project/scRNAseq-MSc-Analysis/upstream_analysis/crypt/differential_expression_scanpy/differential_expression_cluster_Enterocyte Progenitor.csv


/home/glennrdx/virtual_envs/.venv/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:645: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


Saved results for cluster Goblet to /home/glennrdx/Documents/Research_Project/scRNAseq-MSc-Analysis/upstream_analysis/crypt/differential_expression_scanpy/differential_expression_cluster_Goblet.csv


/home/glennrdx/virtual_envs/.venv/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:645: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


...

Saved results for cluster Not Annotated to /home/glennrdx/Documents/Research_Project/scRNAseq-MSc-Analysis/upstream_analysis/crypt/differential_expression_scanpy/differential_expression_cluster_Not Annotated.csv


/home/glennrdx/virtual_envs/.venv/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:645: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


Saved results for cluster Paneth to /home/glennrdx/Documents/Research_Project/scRNAseq-MSc-Analysis/upstream_analysis/crypt/differential_expression_scanpy/differential_expression_cluster_Paneth.csv


/home/glennrdx/virtual_envs/.venv/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:645: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


Saved results for cluster Paneth Progenitor to /home/glennrdx/Documents/Research_Project/scRNAseq-MSc-Analysis/upstream_analysis/crypt/differential_expression_scanpy/differential_expression_cluster_Paneth Progenitor.csv
